In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import sys, os, warnings

sys.path.append(os.path.abspath("/work/bb1018/b380873/tropic_vis/utilities/"))
from plotting_utilities import *
farbe = sim_colors()

# Figure S1: Probability distributions of temperature and RHi along the trajectories

In [ ]:
# Linear T bins in K
T_bins = np.linspace( 190, 240, 50 )
T_bins_c = ( T_bins[1:] + T_bins[:-1] )/2.

# Linear RHi bins in %
RHi_bins = np.linspace( 60, 120, 50 )
RHi_bins_c = ( RHi_bins[1:] + RHi_bins[:-1] )/2.

In [ ]:
basedir = '/work/bb1018/b380873/traj_output/traj_pp/'
names = [ 'qih-Nih-CLAMS-Tf_0V1M0A0R.nc', 'qih-Nih-ICON_0V1M0A0R.nc', 
          'qih-Nih-CLAMS-Tf_0V2M0A0R.nc', 'qih-Nih-ICON_0V2M0A0R.nc',
          'qih-Nih-CLAMS_0V2M0A0R.nc', 'qih-Nih-CLAMS-Tf_0V2M0A0R_noSHflux.nc']
#names = [ 'qih-Nih-CLAMS-Tf_0V1M0A0R_filter.nc', 'qih-Nih-ICON_0V1M0A0R_filter.nc', 
#          'qih-Nih-CLAMS-Tf_0V2M0A0R_filter.nc', 'qih-Nih-ICON_0V2M0A0R_filter.nc',
#          'qih-Nih-CLAMS_0V2M0A0R.nc', 'qih-Nih-CLAMS-Tf_0V2M0A0R_noSHflux.nc']
datasets = []
for n in names:
    d = xr.open_dataset( basedir + n )
    datasets.append( d )

In [ ]:
# Plot the histograms of qi and Ni for extracted trajectories
lw = 2
fs = 18
a = 0.5
suffix = '' # '_outflow', '_insitu', '_flight'
couleurs = [ 'orange', 'orange', 'blue', 'blue', 'blue', 'k' ]
stil = [ '-', '--', '-', '--', ':', '-' ]
lbls = [ r'$\bf{C}$' + '_1M1T1S', r'$\bf{I}$' + '_1M0T0S', r'$\bf{C}$' + '_2M1T1S',
         r'$\bf{I}$' + '_2M0T0S', r'$\bf{C}$' + '_2M0T1S', r'$\bf{C}$' + '_2M0T0S' ]

fig, ax = plt.subplots( nrows=1, ncols=2, figsize=(13.5,6) )
for c, s, d, l, j in zip(couleurs, stil, datasets, lbls, np.arange(6)):
    ax[0].plot( T_bins_c, d['Th'], color=c, linestyle=s, linewidth=lw, label=l )
    #Tm, _ = centroids( 'T' + suffix, j )
    #ax[0].plot( [Tm, Tm], [0, 9], color=c, linestyle=s, linewidth=lw, alpha=a )
    ax[1].plot( RHi_bins_c, d['RHih'], color=c, linestyle=s, linewidth=lw, label=l )
    #RHim, _ = centroids( 'RHi' + suffix, j )
    #ax[1].plot( [RHim, RHim], [0, 9], color=c, linestyle=s, linewidth=lw, alpha=a )
    
ax[0].set_xlim([T_bins[0], T_bins[-1]])
ax[0].set_ylim([0,15])
ax[0].set_xlabel( 'Temperature [K]', fontsize=fs )
ax[0].set_ylabel( 'Probability [%]', fontsize=fs )
ax[0].text(0.02, 0.95, r'$\bf{(a)}$', fontsize=fs+4, transform=ax[0].transAxes)
l = ax[0].legend( loc='upper center', fontsize=fs, frameon=False )
sexy_axes( ax[0], fs )
 
ax[1].set_ylim([0,15])
ax[1].text(0.02, 0.95, r'$\bf{(b)}$', fontsize=fs+4, transform=ax[1].transAxes)
ax[1].set_xlabel( r'RH$_i$ [%]', fontsize=fs )
sexy_axes( ax[1], fs )

#fig.savefig( 'output/CLaMS-ICON-T-RHi-pdfs' + suffix + '.pdf',bbox_inches='tight' )
plt.show()